In [2]:
%load_ext autoreload
%autoreload 2

import nltk
from data_process import replace_notation, remove_notation
import sqlite3 as db
import pandas as pd
import spacy
from spacy.symbols import ORTH
import data_process as dp
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
con = db.connect("./chess.db")
english_annotated_moves = pd.read_sql_query("SELECT * FROM english_annotated_moves", con)
english_annotated_moves

,index,position,move,comment,halfmove_number,game_id,sentiment,auto_sentiment
0,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,e2e4,This is my first gameknot game against someone...,1,44,-1,0
1,1,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,b1c3,"I've been playing the Vienna Gambit as white, ...",3,44,-1,0
2,2,rnbqkbnr/pppp1ppp/8/4p3/4P3/2N5/PPPP1PPP/R1BQK...,f8c5,Minor disappointment.,4,44,-1,0
3,3,r1bqk2r/pppp1ppp/2n2n2/2b1p3/2B1P3/2NP4/PPP2PP...,f2f4,"My idea here is to expand on the kingside, dri...",9,44,-1,0
4,4,r1bqk2r/ppp2ppp/3p1n2/n1b1pP2/2B1P3/2NP4/PPP3P...,d1f3,"Maybe this isn't the greatest plan, since with...",13,44,-1,0
...,...,...,...,...,...,...,...,...
326634,326634,rnb5/p2p1P1N/7p/1pR5/6P1/4k3/2P1B3/3Q1RK1 b - ...,b5b4,Pawn,86,70387,-1,0
326635,326635,rnb5/p2p1P1N/7p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 w -...,f7f8q,Check mate in two moves,87,70387,-1,0
326636,326636,rnb2Q2/p2p3N/7p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 b -...,d7d6,Pawn,88,70387,-1,0
326637,326637,rnb2Q2/p6N/3p3p/2R5/1p4P1/4k3/2P1B3/3Q1RK1 w -...,f8e8,Check,89,70387,-1,0


In [10]:
comments = list(english_annotated_moves.comment)
sentiments = list(english_annotated_moves.sentiment)

In [11]:
tc = comments[:2000]
nlp = spacy.load("en_core_web_sm", disable = ['parser','ner'])

In [12]:
%%time
_ = [nlp(t) for t in tc]

CPU times: user 5.29 s, sys: 1.84 ms, total: 5.29 s
Wall time: 5.3 s


In [15]:
%%time
_ = [t for t in nlp.pipe(tc)]

CPU times: user 2.2 s, sys: 22.8 ms, total: 2.23 s
Wall time: 2.23 s


In [19]:
%%time
_ = [t for t in nlp.pipe(comments, n_process=8)]

CPU times: user 2min 48s, sys: 3.86 s, total: 2min 51s
Wall time: 2min 53s


In [20]:
%%time
_ = [t for t in nlp.pipe(comments, n_process=16, batch_size=128)]

CPU times: user 2min 39s, sys: 4.87 s, total: 2min 44s
Wall time: 2min 45s


In [11]:
# tokenized = [dp.process_comment(com) for com in comments]
tokenized = []
for com in tqdm(comments):
    tokenized.append(dp.process_comment(com))

100%|██████████| 326639/326639 [12:19<00:00, 441.96it/s]


In [28]:
test = []
for com in tqdm(comments):
    test.append(dp.process_comment(com))

100%|██████████| 326639/326639 [00:06<00:00, 46916.58it/s]


In [17]:
tokenized = [list(map(str.lower, com)) for com in tokenized]

In [19]:
vocabulary = set([word for sent in tokenized for word in sent])
len(vocabulary)
# sorted(vocabulary)

36453

In [24]:
for i, com in enumerate(tokenized):
    if "a)4" in com:
        print(i)
        print(comments[i])
        print(com)

107830
Queen's Gambit Accepted, ECO D25, Be6? the book moves are either (a)4 ...g6, 5. Bxc4 Bg7, 6. 0-0 0-0, 7. Nc3 Nfd7. with positional advantage for white (b) 4...a6, 5. Bxc4 b5, 6. Bd3 Bd7, 7. a4 b4. with compensation for material (c) 4...Bg4, 5.Bxc4 e6, 6. h3 Bh5 (alternatively 6. Qb3 Bxf3, 7. gxf3 Nbd7, 8. Qxb7 c5). Here it is apparent that the pawn at c4 is being protected.
['queen', 'gambit', 'accepted', 'eco', 'd25', '__notation__', 'the', 'book', 'move', 'be', 'either', 'a)4', '__notation__', 'with', 'positional', 'advantage', 'for', 'white', 'b', '__notation__', 'with', 'compensation', 'for', 'material', 'c', '__notation__', 'alternatively', '__notation__', 'here', 'it', 'be', 'apparent', 'that', 'the', 'pawn', 'at', '__notation__', 'be', 'be', 'protect']


In [7]:
import re

c = comments[107830]
print(c)
c = dp.process_comment(c)
print(c)
punctuation_regex = r"[\"#\$%&\'\(\)\*\+,-\./:;<=>\@\[\\\]\^_`{\|}~ 0-9]+"
c2 = [t for t in c if not re.search(punctuation_regex, t)]
print(c2)

Queen's Gambit Accepted, ECO D25, Be6? the book moves are either (a)4 ...g6, 5. Bxc4 Bg7, 6. 0-0 0-0, 7. Nc3 Nfd7. with positional advantage for white (b) 4...a6, 5. Bxc4 b5, 6. Bd3 Bd7, 7. a4 b4. with compensation for material (c) 4...Bg4, 5.Bxc4 e6, 6. h3 Bh5 (alternatively 6. Qb3 Bxf3, 7. gxf3 Nbd7, 8. Qxb7 c5). Here it is apparent that the pawn at c4 is being protected.
['queen', 'gambit', 'accept', 'eco', 'd', '__notation__', 'the', 'book', 'move', 'be', 'either', 'a', '__notation__', 'with', 'positional', 'advantage', 'for', 'white', 'b', '__notation__', 'with', 'compensation', 'for', 'material', 'c', '__notation__', 'alternatively', '__notation__', 'here', 'it', 'be', 'apparent', 'that', 'the', 'pawn', 'at', '__notation__', 'be', 'be', 'protect']
['queen', 'gambit', 'accept', 'eco', 'd', 'the', 'book', 'move', 'be', 'either', 'a', 'with', 'positional', 'advantage', 'for', 'white', 'b', 'with', 'compensation', 'for', 'material', 'c', 'alternatively', 'here', 'it', 'be', 'apparent

In [20]:
sorted(vocabulary)

['\x04',
 '\t',
 '\t\t',
 '\t ',
 '\t \t',
 '\t \t \t',
 '\r ',
 '\r \r ',
 '\r \r \r ',
 '\r \r \r \r ',
 '!',
 '!!',
 '!!!',
 '!!!!',
 '?',
 '??',
 '????',
 '__notation__',
 'a',
 'a$$',
 'a$$.',
 'a%u2026',
 'a)4',
 'a)defend',
 'a)get',
 'a)r.',
 'a+b',
 'a-',
 'a-1',
 'a-7',
 'a-8',
 'a.',
 'a.b.',
 'a.k.a',
 'a.k.a.',
 'a.k.a.-',
 'a.karpov',
 'a.m.',
 'a.n.',
 'a.s.a.p',
 'a00',
 'a01',
 'a03',
 'a04',
 'a05',
 'a1',
 'a10',
 'a11',
 'a12',
 'a15',
 'a2',
 'a21',
 'a22',
 'a25',
 'a2]12',
 'a3',
 'a30',
 'a34',
 'a35',
 'a4',
 'a40',
 'a41',
 'a45',
 'a46',
 'a48',
 'a5',
 'a50',
 'a55',
 'a56',
 'a57',
 'a6',
 'a60',
 'a65',
 'a69',
 'a7',
 'a70',
 'a8',
 'a80',
 'a85',
 'a_random_guy',
 'aa',
 'aaaaaaaaaaaaaaaaaaaaaaaargh',
 'aaaaaaaahhhh',
 'aaaaaahhhhhh',
 'aaaaaand',
 'aaaaaggghhhhhh',
 'aaaaah',
 'aaaaahhhhh',
 'aaaaalllllllllll',
 'aaaaand',
 'aaaaargh',
 'aaaah',
 'aaaahhhh',
 'aaaahhhhh',
 'aaaarcg',
 'aaaargh',
 'aaaghh',
 'aaaghhh',
 'aaah',
 'aaahhh',
 'aaahhhhh',
 '

In [8]:
"!!" in vocabulary

False

In [9]:
len(vocabulary)

57274

In [10]:
a = {1,2,3}
list(map(lambda x: x+1, a))

[2, 3, 4]

In [11]:
stemmer = nltk.stem.PorterStemmer()
stemmed = set(list(map(stemmer.stem, vocabulary)))

In [12]:
len(stemmed)


35747

In [13]:
"'s" in vocabulary

True

In [14]:
stemmer.stem("eating")

'eat'

In [15]:
com = replace_notation("(Qg8-b3 Qd2-c3 Qb3-a2+ Rc1-c2 Qa2-b1 Rc2-c1 a5xb4 a3xb4 Qb1-a2+ Rc1-c2 Qa2-d5 Ng1-e2 Rg6-g8 Kh2-g1 Rg8-a8 Qc3-b2 Qd5-f7 Be1-c3 Ra8-a4 Bc3-e1 Qf7-e6 d4-d5 Qe6xd5 Rc2-d2 Qd5-f7 Rd2-d7) +0.89/25 6")
# nltk.word_tokenize(com)
com

'(__variant__)  '

In [1]:
com = remove_notation("?The strategic line is Nb4 -> Qf7 -> Nxd5 -> Qxd5, giving back some material and establishing three free pawns. 23.f4 Nxd3 24.Qxd3 Qf6 25.Bg5 Bf5 26.Bxf6 Bxd3 27.fe de 28.Bxe5/Rxe5 - ...but, alas, as we will see g6 needed protection!")
com
nltk.tokenize.word_tokenize(com)

NameError: name 'remove_notation' is not defined

In [21]:



nlp = spacy.load("en_core_web_sm")

nlp.tokenizer.add_special_case("__variant__", [{ORTH: "__variant__"}])
nlp.tokenizer.add_special_case("__move__", [{ORTH: "__move__"}])

# Process whole documents
text = remove_notation("?? The strategic line is Nb4 -> Qf7 -> Nxd5 -> Qxd5, giving back some material and establishing three free pawns. 23.f4 Nxd3 24.Qxd3 Qf6 25.Bg5 Bf5 26.Bxf6 Bxd3 27.fe de 28.Bxe5/Rxe5 - ...but, alas, as we will see g6 needed protection!")
print(text)


doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
# for entity in doc.ents:
#     print(entity.text, entity.label_)
print([token for token in doc])
print([token.lemma_ for token in doc])

?? The strategic line is  ->  ->  -> , giving back some material and establishing three free pawns. 27.fe de / - ...but, alas, as we will see  needed protection!
Noun phrases: ['The strategic line', 'some material', 'three free pawns', 'we', 'needed protection']
Verbs: ['give', 'establish', 'see', 'need']
[?, ?, The, strategic, line, is,  , -, >,  , -, >,  , -, >, ,, giving, back, some, material, and, establishing, three, free, pawns, ., 27.fe, de, /, -, ..., but, ,, alas, ,, as, we, will, see,  , needed, protection, !]
['?', '?', 'the', 'strategic', 'line', 'be', ' ', '-', '>', ' ', '-', '>', ' ', '-', '>', ',', 'give', 'back', 'some', 'material', 'and', 'establish', 'three', 'free', 'pawn', '.', '27.fe', 'de', '/', '-', '...', 'but', ',', 'alas', ',', 'as', 'we', 'will', 'see', ' ', 'need', 'protection', '!']
